In [1]:
%matplotlib widget

In [2]:
import pickle as pkl
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances
import copy, csv
from rdkit import Chem 
from rdkit.Chem import AllChem
import rdkit.Geometry as Geometry
import pdb
import pickle as pkl
import os
import getpass
import logging

import numpy

In [6]:
orig_coords = numpy.array([[-0.01443023, -3.0990803 ,  1.3292968 ],
       [ 0.10430805, -1.8171468 ,  0.95352185],
       [-0.25510424, -1.348038  , -0.2800352 ],
       [-0.30287   , -0.12065077, -0.783181  ],
       [-0.23225069,  1.0987992 , -0.25925   ],
       [-0.9856278 ,  1.8259783 ,  0.4598882 ],
       [-0.026949  ,  1.3715398 ,  1.1998974 ],
       [ 0.10771976,  0.242562  ,  2.2971797 ],
       [ 0.09727437, -0.98582107,  2.0718298 ]], dtype='float32')

# orig_coords = numpy.array([[-0.38866943, -2.3728359 ,  2.1966689 ],
#        [-0.38858193, -1.8806777 ,  1.3422487 ],
#        [-0.09767051, -0.46843705,  0.8001877 ],
#        [ 0.9831269 , -0.42407602, -0.24033612],
#        [-0.04255892,  0.06275284, -1.1687484 ],
#        [-1.1457566 ,  0.14359272,  0.06276745],
#        [-1.9038742 ,  0.08864039, -0.02204549],
#        [ 0.2188501 ,  0.45626992,  1.9746497 ],
#        [ 0.47660232,  1.2036736 ,  2.8962398 ]], dtype='float32')

# orig_coords = numpy.array([[-0.4544983 , -3.3881435 ,  0.9875389 ],
#        [-0.08907399, -2.5260272 ,  1.1621164 ],
#        [-0.03031953, -1.1649685 ,  1.0085576 ],
#        [-0.2707533 , -0.83433425, -0.3271659 ],
#        [-0.3452333 ,  0.27908975, -0.8893958 ],
#        [-0.2846185 ,  1.3445294 , -0.384991  ],
#        [-0.10835311,  1.7497778 ,  0.9779881 ],
#        [ 0.04251341,  0.9407583 ,  2.0830538 ],
#        [ 0.04075948, -0.4349826 ,  2.0588734 ]], dtype='float32')

# coords = numpy.concatenate([coords, numpy.zeros((9, 3))], axis=0)
# coords += numpy.random.randn(coords.shape[0], 3)


In [20]:
n_trials = 10
n_succ = 0
n_succ_mmff = 0

coords_all = []
mmff_coords_all = []


for ii in range(n_trials):
    coords = orig_coords# + numpy.random.randn(coords.shape[0], 3)


    # mol_ref = Chem.MolFromSmiles('C[C@@H]1C=C[C@@H]2O[C@@H]2CO1')
    # mol_ref = Chem.MolFromSmiles('C[C]1C=C[C]2O[C]2CO1')
    # mol_ref = Chem.MolFromSmiles('N#C[C@H]1CN=CNC=N1')
    mol_ref = Chem.MolFromSmiles('C#C[C@]1(CO)CC[C@H]1O')
    
    mol_ref = Chem.RemoveHs(mol_ref)

    n_est = mol_ref.GetNumAtoms()
    coord_map = {}
    for k in range(n_est):
        point_k = Geometry.Point3D(*coords[k,:].tolist())
        coord_map[k] = point_k

    mol_init_1=Chem.AddHs(mol_ref)
    
#     print(Chem.MolToSmiles(mol_init_1))

#     n_hs = mol_init_1.GetNumAtoms() - mol_ref.GetNumAtoms()
#     for k in range(n_hs):
#         point_k = Geometry.Point3D(*numpy.random.randn(3).tolist())
#         coord_map[n_est+k] = point_k

#     mol_init_1 = copy.deepcopy(mol_ref)

    mol_init_1.RemoveConformer(0)

    ret = AllChem.EmbedMolecule(mol_init_1, 
                                coordMap=coord_map, 
                                useRandomCoords=True,
                                enforceChirality=False,
                                ignoreSmoothingFailures=True)
#     print('return=',ret)
    
    if ret == 0:
        n_succ = n_succ + 1
        coords_all.append(mol_init_1.GetConformer().GetPositions())
        
    mol_baseMMFF = copy.deepcopy(mol_init_1)
    mmff_out = AllChem.MMFFOptimizeMolecule(mol_baseMMFF, maxIters=1000)
    if mmff_out == 0:
        n_succ_mmff = n_succ_mmff + 1
        mol_baseMMFF=Chem.RemoveHs(mol_baseMMFF)
        mmff_coords_all.append(mol_baseMMFF.GetConformer().GetPositions())

        
print ('DG: {}/{}'.format(n_succ, n_trials))
print ('MMFF: {}/{}'.format(n_succ_mmff, n_trials))

DG: 10/10
MMFF: 10/10


In [67]:
mol_ref.GetNumAtoms(), mol_init_1.GetNumAtoms(), n_est

(9, 19, 9)

In [17]:
y = orig_coords

In [18]:
x = mol_init_1.GetConformer()

In [19]:
x.GetPositions()[:9,:], y

(array([[ 8.01706628e-01,  1.38952459e+00,  1.27302612e+00],
        [ 5.72904114e-01,  1.67856758e-01,  7.62882958e-01],
        [ 3.57152978e-01, -1.03344293e+00,  3.42439231e-01],
        [-1.05794534e+00, -1.26751341e+00, -1.43396216e-03],
        [-1.03406103e+00, -2.38272419e+00, -7.21037948e-01],
        [ 1.23053012e-01, -2.41094042e+00, -1.69179440e+00],
        [-2.02429449e-01, -1.23456028e+00, -2.18952153e+00],
        [ 7.84699288e-01,  2.54078007e-02, -1.66547028e+00],
        [ 4.11567488e-01,  8.41757253e-01, -8.06911793e-01]]),
 array([[-0.01443023, -3.0990803 ,  1.3292968 ],
        [ 0.10430805, -1.8171468 ,  0.95352185],
        [-0.25510424, -1.348038  , -0.2800352 ],
        [-0.30287   , -0.12065077, -0.783181  ],
        [-0.23225069,  1.0987992 , -0.25925   ],
        [-0.9856278 ,  1.8259783 ,  0.4598882 ],
        [-0.026949  ,  1.3715398 ,  1.1998974 ],
        [ 0.10771976,  0.242562  ,  2.2971797 ],
        [ 0.09727437, -0.98582107,  2.0718298 ]], dtype=f

In [20]:
dist = 1./2. * ((orig_coords[:,None,:] - orig_coords[None,:,:]) ** 2).sum(-1)

eigs = numpy.linalg.eigvalsh(dist)

print('# of positive eigenvalues in the distance matrix:', (numpy.linalg.eigvalsh(dist) > 0.).sum())

_, s, _ = numpy.linalg.svd(coords)

print('# of positive singular values in the coordinate matrix:', (s > 0.).sum())

# of positive eigenvalues in the distance matrix: 3
# of positive singular values in the coordinate matrix: 3


In [21]:
coords = x.GetPositions()

dist = 1./2. * ((coords[:,None,:] - coords[None,:,:]) ** 2).sum(-1)

eigs = numpy.linalg.eigvalsh(dist)

print('# of positive eigenvalues in the distance matrix:', (numpy.linalg.eigvalsh(dist) > 0.).sum())

_, s, _ = numpy.linalg.svd(coords)

print('# of positive singular values in the coordinate matrix:', (s > 0.).sum())

# of positive eigenvalues in the distance matrix: 9
# of positive singular values in the coordinate matrix: 3


In [22]:
from matplotlib import pyplot as plot

In [23]:
from mpl_toolkits import mplot3d


In [35]:
plot.figure(figsize=(20,5))

ax = plot.subplot(2,n_trials+1,1, projection='3d')
for jj in range(orig_coords.shape[0]):
    ax.text(orig_coords[jj,0], orig_coords[jj,1], orig_coords[jj,2], '{}'.format(jj))
ax.plot3D(orig_coords[:,0], orig_coords[:,1], orig_coords[:,2], '-')

for tt in range(numpy.minimum(2, n_trials)):
    ax = plot.subplot(2,n_trials+1,tt+2, projection='3d')
    for jj in range(orig_coords.shape[0]):
        ax.text(coords_all[tt][jj,0], coords_all[tt][jj,1], coords_all[tt][jj,2], '{}'.format(jj))
    ax.plot3D(coords_all[tt][:9,0], coords_all[tt][:9,1], coords_all[tt][:9,2], 'r-o')

for tt in range(numpy.minimum(2, n_trials)):
    ax = plot.subplot(2,n_trials+1,n_trials+1+tt+2, projection='3d')
    for jj in range(orig_coords.shape[0]):
        ax.text(mmff_coords_all[tt][jj,0], mmff_coords_all[tt][jj,1], mmff_coords_all[tt][jj,2], '{}'.format(jj))
    ax.plot3D(mmff_coords_all[tt][:9,0], mmff_coords_all[tt][:9,1], mmff_coords_all[tt][:9,2], 'r-o')

    
plot.tight_layout()
plot.show()

FigureCanvasNbAgg()

In [18]:
plot.figure()
ax = plot.axes(projection='3d')

for ii in range(orig_coords.shape[0]):
    xx = numpy.vstack([orig_coords[ii, :], coords[ii,:]])
    ax.plot3D(xx[:,0], xx[:,1], xx[:,2], 'x-')

plot.show()

FigureCanvasNbAgg()

In [32]:
mmff_coords_all[tt].shape, orig_coords.shape

((9, 3), (19, 3))

In [19]:
xx

array([[ 0.47660232,  1.2036736 ,  2.89623976],
       [ 1.5996617 , -1.26414464,  1.33259319]])